# Practice Optimizing Dataframes and Processing in Chunks Using Lending Club's Lending Data


We'll be working with financial lending data from Lending Club, a marketplace for personal loans that matches borrowers with investors. You can read more about the marketplace on its website.

The Lending Club's website lists approved loans. Qualified investors can view the borrower's credit score, the purpose of the loan, and other details in the loan applications. Once a lender is ready to back a loan, it selects the amount of money it wants to fund. When the loan amount the borrower requested is fully funded, the borrower receives the money, minus the origination fee that Lending Club charges.

We'll be working with a dataset of loans approved from 2007-2011, which you can download from Lending Club's website. We've already removed the desc column for you to make our system run more quickly.

If we read in the entire data set, it will consume about 67 megabytes of memory. Let's imagine that we only have 10 megabytes of memory available throughout this project, so you can practice the concepts you learned in the last two missions. 

In [2]:
import pandas as pd
pd.options.display.max_columns = 99


In [3]:
# Samply of the dataset
first_five = pd.read_csv('loans_2007.csv', nrows=5)
first_five

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


### Memory footprint for 3000 rows (in megabytes)

In [4]:
first_n = pd.read_csv('loans_2007.csv', nrows=3000)
print(first_n.memory_usage(deep=True).sum() / 2 ** 20)


4.649013519287109


### How many rows in dataset?

In [5]:
total_rows = 0
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)   
for chunk in chunk_iter:
    total_rows += len(chunk)
    
print(total_rows)


42538


### Explore Data in the Chunk

#### How many columns have a numeric type? How many have a string type?

In [6]:
first_five.dtypes.value_counts()


float64    30
object     21
int64       1
dtype: int64

In [7]:
numeric_cols_num = len(first_five.select_dtypes(include=['float64', 'int64']).columns)   
print('Numeric columns number: {}'.format(numeric_cols_num))

string_cols_num = len(first_five.select_dtypes(include='object').columns)                 
print('String columns number: {}'.format(string_cols_num))



Numeric columns number: 31
String columns number: 21


In [8]:
# total rows by column
total_rows_by_col = {}
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)   
for chunk in chunk_iter:
    for col in chunk:
        if col in total_rows_by_col:
            total_rows_by_col[col] += len(chunk[col])
        else:
            total_rows_by_col[col] = len(chunk[col])
            
print(total_rows_by_col)            
    

{'grade': 42538, 'acc_now_delinq': 42538, 'inq_last_6mths': 42538, 'pymnt_plan': 42538, 'open_acc': 42538, 'purpose': 42538, 'revol_bal': 42538, 'sub_grade': 42538, 'total_rec_int': 42538, 'zip_code': 42538, 'chargeoff_within_12_mths': 42538, 'loan_status': 42538, 'last_credit_pull_d': 42538, 'last_pymnt_amnt': 42538, 'emp_length': 42538, 'revol_util': 42538, 'recoveries': 42538, 'out_prncp': 42538, 'collection_recovery_fee': 42538, 'total_rec_prncp': 42538, 'policy_code': 42538, 'int_rate': 42538, 'total_rec_late_fee': 42538, 'verification_status': 42538, 'issue_d': 42538, 'addr_state': 42538, 'funded_amnt': 42538, 'member_id': 42538, 'loan_amnt': 42538, 'collections_12_mths_ex_med': 42538, 'home_ownership': 42538, 'pub_rec_bankruptcies': 42538, 'delinq_amnt': 42538, 'total_acc': 42538, 'id': 42538, 'initial_list_status': 42538, 'application_type': 42538, 'pub_rec': 42538, 'last_pymnt_d': 42538, 'funded_amnt_inv': 42538, 'annual_inc': 42538, 'total_pymnt': 42538, 'total_pymnt_inv': 42

#### How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?


In [9]:
# unique values in each string column
obj_col_vc = {} # key is the column name
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)   
for chunk in chunk_iter:
    obj_cols = chunk.select_dtypes(include='object')
    #print(obj_cols.columns)
    for col in obj_cols.columns:
        current_vc = obj_cols[col].value_counts()
        if col in obj_col_vc:
            obj_col_vc[col].append(current_vc) 
        else:
            obj_col_vc[col] = [current_vc]
    
# combine
final_vcs = {}
vc_lt50pct = []
for col in obj_col_vc:
    combined_vc = pd.concat(obj_col_vc[col])
    final_vc = combined_vc.groupby(combined_vc.index).sum()
    final_vcs[col] = final_vc
    
    #if final_vc[col].sum() < total_rows_by_col[col] * 0.5:
    #    vc_lt50pct.append(col)

print(final_vcs['loan_status'].sum())
print
#print(vc_lt50pct)
    

42535


<function print>

#### Which float columns have no missing values and could be candidates for conversion to the integer type?

In [10]:
float_cols_mv = {}
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)   
for chunk in chunk_iter:
    float_cols = chunk.select_dtypes(include='float64')
    for col in float_cols:
        if col in float_cols_mv:
            float_cols_mv[col] += float_cols[col].isnull().sum()  
        else:
            float_cols_mv[col] = float_cols[col].isnull().sum()  

print(float_cols_mv)


{'loan_amnt': 3, 'acc_now_delinq': 32, 'total_pymnt_inv': 3, 'inq_last_6mths': 32, 'collections_12_mths_ex_med': 148, 'pub_rec_bankruptcies': 1368, 'open_acc': 32, 'revol_bal': 3, 'total_rec_int': 3, 'total_acc': 32, 'policy_code': 3, 'chargeoff_within_12_mths': 148, 'pub_rec': 32, 'recoveries': 3, 'funded_amnt_inv': 3, 'last_pymnt_amnt': 3, 'out_prncp': 3, 'collection_recovery_fee': 3, 'total_rec_prncp': 3, 'total_pymnt': 3, 'total_rec_late_fee': 3, 'out_prncp_inv': 3, 'annual_inc': 7, 'dti': 3, 'tax_liens': 108, 'delinq_2yrs': 32, 'funded_amnt': 3, 'member_id': 3, 'delinq_amnt': 32, 'installment': 3}


In [11]:
float_cols_no_mv = [k for (k, v) in float_cols_mv.items() if v==0]   
float_cols_no_mv


[]

#### Calculate the total memory usage across all of the chunks.


In [28]:
# in megabytes
memory_usages_before = []
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)   
for chunk in chunk_iter:
    #memory_usages.append(chunk.memory_usage(deep=True).sum()/ 2**20)    
    memory_usage = chunk.memory_usage(deep=True).sum() / 2**20 
    memory_usages_before.append(memory_usage)
    
total_memory_usages_before = sum(memory_usages_before)   
print('Total Memory Usage Across all the Chunks before cleaning: {:.3f} megabytes'.format(total_memory_usages_before) ) 

    

Total Memory Usage Across all the Chunks before cleaning: 66.215 megabytes


### Data Cleaning and calculate memory usage after cleaning



Convert 'issue_d', 'earliest_cr_line', 'last_pymnt_d', and 'last_credit_pull_d' to datetime.

Determine which string columns you can convert to a numeric type if you clean them.  For example, column 'term' data has month at the end.  The 'int_rate' and 'revol_util' column is only a string because of the % sign at the end.  

Convert the columns that contain numeric values to the float type.  Focus on columns that would actually be useful for analysis and modelling.

Determine which columns have a few unique values and convert them to the category type.  For example, you may want to convert the grade and sub_grade columns.


#### Based on your conclusions, perform the necessary type changes across all chunks. Calculate the total memory footprint, and compare it with the previous one.

In [13]:

convert_col_dtypes = {
    "sub_grade": "category", 
    "home_ownership": "category", 
    "verification_status": "category", 
    "purpose": "category"
}

In [29]:
# Convert issue_d, earliest_cr_line, last_pymnt_d, and last_credit_pull_d to datetime.¶
# Convert 'term', 'int_rate' and 'revol_util' to numerical by data cleaning.
# calculate missing value counts for float columns
# convert 'grade', 'sub_grade' to category type
chunk_iter = pd.read_csv('loans_2007.csv', 
                         chunksize=3000, 
                         dtype = convert_col_dtypes,
                         parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"]   
                        ) 

float_mv_counts_by_col = {}
memory_usages_after = []

for chunk in chunk_iter:
    term_cleaned = chunk['term'].str.strip(' ').str.rstrip('months')  
    chunk['term'] = pd.to_numeric(term_cleaned)
    
    revol_cleaned = chunk['revol_util'].str.rstrip('%')
    chunk['revol_util'] = pd.to_numeric(revol_cleaned)
    int_rate_cleaned = chunk['int_rate'].str.rstrip('%') 
    chunk['int_rate'] = pd.to_numeric(int_rate_cleaned)
    
    memory_usage = chunk.memory_usage(deep=True).sum() / 2**20  
    memory_usages_after.append(memory_usage)   
 
    float_cols = chunk.select_dtypes(include='float')
    for col in float_cols.columns:
        mv_count = len(chunk) - chunk[col].count()
        if col in float_mv_counts_by_col:
            float_mv_counts_by_col[col] += mv_count
        else: 
            float_mv_counts_by_col[col] = mv_count

print(chunk.info())

print('Missing Value Count for float columns by column:'.format(float_mv_counts_by_col))  

print('Total memory usage after cleaning and converting: {:.3f} megabytes'.format(sum(memory_usages_after)))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538 entries, 42000 to 42537
Data columns (total 52 columns):
id                            538 non-null object
member_id                     536 non-null float64
loan_amnt                     536 non-null float64
funded_amnt                   536 non-null float64
funded_amnt_inv               536 non-null float64
term                          536 non-null float64
int_rate                      536 non-null float64
installment                   536 non-null float64
grade                         536 non-null object
sub_grade                     536 non-null category
emp_title                     499 non-null object
emp_length                    536 non-null object
home_ownership                536 non-null category
annual_inc                    532 non-null float64
verification_status           536 non-null category
issue_d                       536 non-null datetime64[ns]
loan_status                   536 non-null object
pymnt_plan       

It looks like we were able to realize some powerful memory savings of about 40% memory usage by converting to the category type and converting string columns to numeric ones.



### optimize the numeric columns using the pandas.to_numeric() function.

While working with dataframe chunks:
- Identify float columns that contain missing values, and that we can convert to a more space efficient subtype.
- Identify float columns that don't contain any missing values, and that we can convert to the integer type because they represent whole numbers.
- Based on your conclusions, perform the necessary type changes across all chunks. Calculate the total memory footprint and compare it with the previous one.

### Create a function to automates cleaning work

Create a function that automates as much of the work you just did as possible, so that you could use it on other Lending Club data sets. This function should:
- Determine the optimal chunk size based on the memory constraints you provide
- Determine which string columns can be converted to numeric ones by removing the % character
- Determine which numeric columns can be converted to more space efficient representations

In [1]:
first_five[useful_obj_cols].info()

NameError: name 'first_five' is not defined

In [ ]:
useful_obj_cols = ['term', 'sub_grade', 'emp_title', 'home_ownership', 'verification_status', 'issue_d', 'purpose', 'earliest_cr_line', 'revol_util', 'last_pymnt_d', 'last_credit_pull_d']   
